<a href="https://colab.research.google.com/github/buddypia/openai-agents-sdk-colab/blob/master/agent_lifecycle_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Pythonパッケージインストール
!pip install openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.6 MB/s eta 0:00:00


In [20]:
import os
from google.colab import userdata

# ColabのシークレットからOpenAIのAPIキーを取得
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

# OpenAIエージェントにAPIキーを設定
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [21]:
from typing import Any

from agents import Agent, AgentHooks, RunContextWrapper, Tool

class CustomAgentHooks(AgentHooks):
    def __init__(self, display_name: str):
        self.event_counter = 0
        self.display_name = display_name

    # エージェント開始時に呼び出されるフック
    async def on_start(self, context: RunContextWrapper, agent: Agent) -> None:
        self.event_counter += 1
        print(f"### ({self.display_name}) {self.event_counter}: エージェント {agent.name} が開始しました")

    # エージェント終了時に呼び出されるフック
    async def on_end(self, context: RunContextWrapper, agent: Agent, output: Any) -> None:
        self.event_counter += 1
        print(
            f"### ({self.display_name}) {self.event_counter}: エージェント {agent.name} が出力 {output} で終了しました"
        )

    # エージェント間で処理が引き継がれる (ハンドオフ) 時に呼び出されるフック
    async def on_handoff(self, context: RunContextWrapper, agent: Agent, source: Agent) -> None:
        self.event_counter += 1
        print(
            f"### ({self.display_name}) {self.event_counter}: エージェント {source.name} が {agent.name} に処理を引き継ぎました"
        )

    # エージェントがツールを開始する時に呼び出されるフック
    async def on_tool_start(self, context: RunContextWrapper, agent: Agent, tool: Tool) -> None:
        self.event_counter += 1
        print(
            f"### ({self.display_name}) {self.event_counter}: エージェント {agent.name} がツール {tool.name} を開始しました"
        )

    # エージェントがツールを終了した時に呼び出されるフック
    async def on_tool_end(
        self, context: RunContextWrapper, agent: Agent, tool: Tool, result: str
    ) -> None:
        self.event_counter += 1
        print(
            f"### ({self.display_name}) {self.event_counter}: エージェント {agent.name} がツール {tool.name} を結果 {result} で終了しました"
        )

In [22]:
import random

from agents import function_tool

# 乱数生成ツール
@function_tool
def random_number(max: int) -> int:
    """
    指定された最大値までの乱数を生成します。
    """
    return random.randint(0, max)

# 2倍乗算ツール
@function_tool
def multiply_by_two(x: int) -> int:
    """単純な2倍の乗算。"""
    return x * 2

In [34]:
from pydantic import BaseModel

from agents import Agent

# エージェントの最終的な出力形式を定義するPydanticモデル
class FinalResult(BaseModel):
    number: int

# 2番目のエージェント (数値を2倍する)
multiply_agent = Agent(
    name="Multiply Agent",
    instructions="数値を2倍し、最終結果を返します。",
    tools=[multiply_by_two],
    output_type=FinalResult,
    hooks=CustomAgentHooks(display_name="Multiply Agent"),
)

# 最初のエージェント (乱数を生成し、条件分岐する)
start_agent = Agent(
    name="Start Agent",
    instructions="乱数を生成します。偶数の場合は停止します。奇数の場合は multiply エージェントに処理を引き継ぎます。",
    tools=[random_number],
    output_type=FinalResult,
    handoffs=[multiply_agent],
    hooks=CustomAgentHooks(display_name="Start Agent"),
)

In [24]:
user_input = input("乱数の最大値を入力してください: ")

乱数の最大値を入力してください: 100


In [33]:
from agents import Runner

result = await Runner.run(
        start_agent,
        input=f"0から{user_input}までの乱数を生成してください。",
    )
print(f"完了! result: {result.final_output}")

### (Start Agent) 1: エージェント Start Agent が開始しました
### (Start Agent) 2: エージェント Start Agent がツール random_number を開始しました
### (Start Agent) 3: エージェント Start Agent がツール random_number を結果 64 で終了しました
### (Start Agent) 4: エージェント Start Agent が出力 number=64 で終了しました
完了! result: number=64
